<div align="right">Antoine Moulin</div>

<h1> SD-TSIA211 </h1>
<h1> Computer Lab: Nonnegative Matrix Factorization </h1>

In [4]:
### IMPORTS ###

import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds
from scipy.optimize import check_grad
import time

<h2> 1. Database </h2>

<h3> Question 1.1 </h3>

Download and extract the database of faces, collected by AT&T Laboratories Cambridge,
on https://www.cl.cam.ac.uk/research/dtg/attarchive/facedatabase.html.
How many images are there in the database? How many pixels are there in each image?

In [5]:
def build_matrix_from_faces(folder='att_faces', minidata=False):
    # load images
    # 400 images of size (112, 92)
    M = []
    if minidata is True:
        nb_subjects = 1
    else:
        nb_subjects = 40
    for subject in range(1, nb_subjects + 1):
        for image in range(1, 11):
            face = plt.imread(folder + '/s' + str(subject)
                              + '/' + str(image) + '.pgm')
            M.append(face.ravel())

    return np.array(M, dtype=float)

full_db = build_matrix_from_faces()
mini_db = build_matrix_from_faces(minidata = True)

In [6]:
current_db = mini_db # just change this line to change the db used
n, p = current_db.shape

There are 400 images of 92 x 112 pixels, i.e. 10304 pixels in each image.

<h2> 2 - Presentation of the model </h2>

We have $n$ images with $p$ pixels. We are going to vectorize the images, that is consider them as a mere list of $p$ pixel values. Then, we stack them all in a matrix $M$ of size $n \times p$.

The goal of Nonnegative Matrix Factorization (NMF) is to (approximately) factorize the matrix $M$, which has only nonnegative entries, into the product of two matrices $W$ and $H$ so that $M \approx WH$. $W$ is a $n \times k$ nonnegative matrix and $H$ is a $k \times p$ nonnegative matrix, where $k \leq min(n, p)$ is a parameter of the model called the dimension of the latent space. Each line $H_{j, :}$ of $H$ can be interpreted as the image of a piece of face. The coefficient $W_{i, j}$ tells us which proportion of image $H_{j, :}$ is present in the face $M_{i, :}$.

The model can be trained by solving the optimization problem

$$ \left( \widehat{W}, \widehat{H} \right) \in \arg \min_{W \geq 0, H \geq 0} \frac{1}{2np} \sum_{i=1}^{n} \sum_{l=1}^{p} \left( M_{i, l} - \sum_{j=1}^{k} W_{i, j} H_{j, l} \right)^{2}$$

$$\left( \widehat{W}, \widehat{H} \right) \in \arg \min_{W \geq 0, H \geq 0} \frac{1}{2np} \left|\left| M - WH \right|\right|_{F}^{2} $$

where $\left|\left| \cdot \right|\right|_{F}$ is Frobenius's norm.

When $k = 1$ the solution is proportional to the left and right singular vectors of $M$ associated with the singular value with highest magnitude. In general, we need to use an optimization algorithm.

<h3> Question 2.1 </h3>

Show that the objective function is not convex. Calculate its gradient. Is the gradient Lipschitz continuous?

Let define $g$ the objective function as: 
$$ g : \begin{array} \left \mathbb{R}^{n \times k} \times \mathbb{R}^{k \times p} \longrightarrow \mathbb{R} \\
(W, H) \mapsto \frac{1}{2np} || M - WH ||_{F}^{2} \end{array}$$ and <b>show that $g$ is not convex</b>.

In order to do this, let use the contraposed of the following property:

Let $\alpha \in \mathbb{N} \setminus \lbrace 0, 1 \rbrace$. If $f : \mathbb{R}^{\alpha} \longrightarrow \mathbb{R}$ is convex, then for all $i, j \in [\![ 1, \alpha ]\!]$, $f_{i, j}$ is convex, where $$f_{i, j} : \begin{array} \left \mathbb{R}^{2} \longrightarrow \mathbb{R} \\
(x_{i}, x_{j}) \mapsto f(0, ..., 0, x_{i}, 0, ..., 0, x_{j}, 0, ..., 0) \end{array} $$


Then, we just have to find a partial function of $g$ that is not convex in order to prove that $g$ is not convex (because $g$ can be seen as a function from $\mathbb{R}^{nk + kp}$). Let consider $g_{1, nk + 1}$ (we put every coefficients of $W$ and $H$ to $0$ except the first of each matrix). 

For all $(w, h) \in \mathbb{R}^{2}$, we have $$g_{1, nk + 1}(w, h) = C(M) + \frac{1}{2np}(M_{1, 1} - wh)^{2}$$ where $C(M)$ is a constant depending on the coefficients of M.

Then, proving that $g_{1, nk + 1}$ is not convex is equivalent to prove that $$s : \begin{array} \left  
\mathbb{R} \times \mathbb{R} \longrightarrow \mathbb{R} \\
(w, h) \mapsto (1 - wh)^{2} \end{array}$$ is not convex.

We have $$\left\{\begin{array} 
ss(-1, -1) = 0 \\
s(1, 1) = 0 \\
s \left( \frac{1}{2}(1, 1) + \frac{1}{2}(-1, -1) \right) = s(0, 0) = 1 \end{array}\right.$$

Thus, $$s(0, 0) > \frac{1}{2} s(1, 1) + \frac{1}{2} s(-1, -1)$$
The definition of convexity is not satisfied. $s$ is not convex, so $g_{1, nk+1}$ is not.

<p style="text-align:center";><span style="border:1px solid black;padding:1%">Thus, the objective function $g$ is not convex.</span></p>

<b>Let compute the gradient of $g$</b>. We have, for all $W, u \in \mathbb{R}^{n \times k}$ and $H, v \in \mathbb{R}^{k \times p}$:

$$ \begin{array} gg(W + u, H + v) & = \frac{1}{2np} \langle M - (W + u)(H + v), M - (W + u)(H + v) \rangle_{F} \\
 & = \frac{1}{2np} \langle M - WH - Wv - uH - uv, M - WH - Wv - uH - uv \rangle_{F} \\
 & = \frac{1}{2np} \langle M - WH, M - WH \rangle_{F} - \frac{1}{np} \langle M - WH, Wv + uH \rangle_{F} + o \left((u, v)\right) \\
 & = g(W, H) - \frac{1}{np} \langle M - WH, uH \rangle_{F} - \frac{1}{np} \langle M - WH, Wv \rangle_{F} + o \left((u, v)\right) \\
 & = g(W, H) - \frac{1}{np} \left( \langle (M - WH)H^{T}, u \rangle_{F} - \langle W^{T}(M - WH), v \rangle_{F} \right) + o \left((u, v)\right) \end{array} $$

Then, $$\boxed{\nabla_{W}g(W, H) = - \frac{1}{np} (M - WH)H^{T} \,\, and \,\, \nabla_{H}g(W, H) = - \frac{1}{np} W^{T} (M - WH)}$$

This results could have been obtained directly.

Now, <b>let prove that the gradient is not Lipschitz continuous</b>. 

In what follows we will consider the appropriate Frobenius's norms for all the spaces considered (e.g. for a tuple of matrices $(W,H)$ we will have the sum over all the coefficients of $W$ and $H$). 

Let $W \in \mathbb{R}^{n \times k}, H, H' \in \mathbb{R}^{k \times p}$ and $\alpha > 0$. We have :

$\begin{array} \nabla\nabla_{W} g(W, \alpha H) - \nabla_{W} g(W, \alpha H') & 
= - \frac{1}{np} \left[ \alpha M H^{T} - \alpha^{2} W HH^{T} - \alpha M H'^{T} + \alpha^{2} W H'H'^{T} \right] \\
 & = - \frac{1}{np} \left[ \alpha M \left( H^{T} - H'^{T} \right) + \alpha^{2} W \left( H'H'^{T} - HH^{T} \right) \right] \\
 & = - \frac{\alpha}{np} \left[ M \left( H^{T} - H'^{T} \right) + \alpha W \left( H'H'^{T} - HH^{T} \right) \right] \end{array}$

Then, we have 

$\left|\left| \nabla_{W} g(W, \alpha H) - \nabla_{W} g(W, \alpha H') \right|\right| = \frac{\alpha}{np} \left|\left| M \left( H^{T} - H'^{T} \right) + \alpha W \left( H'H'^{T} - HH^{T} \right) \right|\right|$

and

$\left|\left| (W, \alpha H) - (W, \alpha H') \right|\right| = \alpha \left|\left| (0, H - H') \right|\right|$

As $\alpha > 0$, we can divide by $\alpha$ and then compare the quantities : $$\frac{1}{np} \left|\left| M \left( H^{T} - H'^{T} \right) + \alpha W \left( H'H'^{T} - HH^{T} \right) \right|\right| \,\, and \,\, \left|\left| (0, H - H') \right|\right|$$

For $u, v \in \mathbb{N}$, we denote by $E^{u, v}$ the matrix with size $u \times v$ which has all its coefficients to $0$ except the first one that is equal to $1$.

We take $H = E^{k, p}, H' = 2 E^{k, p}, W = E^{n, k}$. Thus, we have

$$ \frac{1}{np} \left|\left| M \left( H^{T} - H'^{T} \right) + \alpha W \left( H'H'^{T} - HH^{T} \right) \right|\right| = \frac{1}{np} \left[ \left( 3 \alpha - M_{1, 1} \right)^{2} + K(M) \right]$$ 

where $K(M)$ is a constant because $M$ is fixed, and 

$$\left|\left| (0, H - H') \right|\right| = 1$$

As $\frac{1}{np} \left[ \left( 3 \alpha - M_{1, 1} \right)^{2} + K(M) \right] \underset{\alpha \rightarrow + \infty}{\longrightarrow} + \infty$, we showed that :

$$\forall C \geq 0, \exists (x, y), \left|\left| \nabla_{W} g(x) - \nabla_{W} g(y) \right|\right| > C \left|\left| x - y \right|\right|$$

<p style="text-align:center";><span style="border:1px solid black;padding:1%">Finally, $\nabla_{W} g$ is not Lipschitz continuous, that is to say the gradient of $g$ is not Lipschitz continuous.</span></p>

<h2> 3 - Find $W$ when $H_{0}$ is fixed </h2>

<h3> Question 3.1 </h3>

We initialize as follows. What is the advantage of this choice? What would be other possibilities for the initialization?

In [7]:
k = 2
W0, S, H0 = svds(mini_db.astype(float), k)
W0 = np.maximum(0, W0 * np.sqrt(S))
H0 = np.maximum(0, (H0.T * np.sqrt(S)).T)

The advantage of this choice (i.e. using the Singular Value Decomposition) is that, when $M$ is a square matrix that satisfies $Sp \left( M \right) \subset \mathbb{R}_{+}^{*}$, this initialization is a decomposition close to the minimum of the objective function (and the exact minimum if $n = k = p$).

For the initialization, we could take $W_{0} = 0$ and $H_{0} = 0$ but this decomposition is not likely to be as close to the minimum as the above decomposition. 

In a first part, we would like to solve the simpler problem:

$$g \left( W \right) = \frac{1}{2np} \left|\left| M - WH_{0} \right|\right|_{F}^{2}$$

$$W_{1} \in \arg \min_{W \geq 0} g \left( W \right)$$

<h3> Question 3.2 </h3>

Is the objective function $g$ convex? Calculate its gradient. We will admit that the gradient of $g$ is Lipschitz continuous with constant $L_{0} = \frac{1}{np}\|(H_{0})^{T}H_{0}\|_{F}$.

$g$ is a sum of convex quadratic functions. 
<p style="text-align:center";><span style="border:1px solid black;padding:1%">Thus, $g$ is convex.</span></p>

Besides, we have $$\boxed{\nabla_{W} g(W) = - \frac{1}{np} (M - WH_{0})H_{0}^{T}}$$

<h3> Question 3.3 </h3>

Write a function to compute $g(W)$ and another to compute $\nabla g(W)$.
You can check your computations using the function <tt>scipy.optimize.check_grad</tt> (as <tt>check_grad</tt> cannot deal with matrix variable, you may need to vectorize your variables).

In [8]:
def unvectorize_M(W_H, M = mini_db):
    # number of elements in W_H is (n+p)*k where M is of size n x m
    # W has the nk first elements
    # H has the kp last elements
    n, p = M.shape
    k = W_H.shape[0] // (n + p)
    W = W_H[:n * k].reshape((n, k))
    H = W_H[n * k:].reshape((k, p))
    return W, H

def vectorize(W, H):
    return np.concatenate((W.ravel(), H.ravel()))

def compute_obj_function(W_H, M = mini_db):
    W_unvec, H_unvec = unvectorize_M(W_H, M)    
    return (2*M.shape[0]*M.shape[1])**(-1) * np.linalg.norm(M - W_unvec @ H_unvec)**2

def compute_grad_obj_function(W_H, M = mini_db):
    W_unvec, H_unvec = unvectorize_M(W_H, M)
    n, p = M.shape
    grad_W = (- n*p)**(-1) * (M - W_unvec @ H_unvec) @ H_unvec.T
    grad_H = (- n*p)**(-1) * W_unvec.T @ (M - W_unvec @ H_unvec)
    return vectorize(grad_W, grad_H)

In [9]:
check_grad(compute_obj_function, compute_grad_obj_function, vectorize(W0,H0))

0.0007838044905674255

<h3> Question 3.4 </h3>

Let us define the function:
$$ \iota_{\mathbb{R}_{+}} : \mathbb{R} \rightarrow \mathbb{R}\cup\{+\infty\}$$
$$ x \rightarrow \left\{ \begin{matrix} 0 \,\,\,\, si \,\,\,\, x \geq 0 \\ +\infty \,\,\,\, si \,\,\,\, x < 0 \end{matrix} \right. $$ 

Show that for all $\gamma > 0$, $prox_{\gamma \iota_{\mathbb{R_{+}}}}$ is the projection onto $\mathbb{R}_{+}$.

Let $\gamma > 0, x \in \mathbb{R}$. Using the definition, we have :

$prox_{\gamma \iota_{\mathbb{R}_{+}}}(x) = arg\displaystyle\min_{y \in \mathbb{R}} \left( \gamma\iota_{\mathbb{R}_{+}}(y) + \frac{|y - x|^{2}}{2} \right)$

$prox_{\gamma \iota_{\mathbb{R}_{+}}}(x) = arg\displaystyle\min_{y \in \mathbb{R}_{+}} \left( \gamma\iota_{\mathbb{R}_{+}}(y) + \frac{|y - x|^{2}}{2} \right)$ because $\gamma\iota_{\mathbb{R}_{+}}(y) = + \infty$ if $y < 0$.

$prox_{\gamma \iota_{\mathbb{R}_{+}}}(x) = arg\displaystyle\min_{y \in \mathbb{R}_{+}} \left( |y - x| \right)$ because $\iota_{\mathbb{R}_{+}}(y) = 0$ for $y \geq 0$.

$prox_{\gamma \iota_{\mathbb{R}_{+}}}(x) = \left\{ \begin{array} xx \,\, si \,\, x \geq 0 \\
0 \,\, si \,\, x < 0 \end{array} \right.$

This is the point of $\mathbb{R}_{+}$ closest to $x$, i.e. $proj_{\mathbb{R}_{+}}(x)$. Thus,

$$\boxed{\forall \gamma > 0, prox_{\gamma \iota_{\mathbb{R}_{+}}} = proj_{\mathbb{R}_{+}}}$$

<h3> Question 3.5 </h3>

Code a function <tt>projected\_gradient\_method(val\_g, grad\_g, W0, gamma, N)</tt> that minimizes a function $g$ subject to nonnegativity constraints by the projected gradient method with a constant step size $\gamma$, starting from $W_{0}$ and stopping after $N$ iterations.

In [10]:
def projected_gradient_method(val_g, grad_g, W0, gamma, N):
    x = W0
    for k in range(N):
        x = np.maximum(0, x - gamma * unvectorize_M(grad_g(vectorize(x, H0)))[0])
    return val_g(vectorize(x, H0))

<h3> Question 3.6 </h3>

Use the function to minimize $g$ with $N = 100$.

In [25]:
gamma = 1 / (np.linalg.norm(H0.T.dot(H0), 'fro') / (n * p))

t1_proj = time.time()
min_g_proj = projected_gradient_method(compute_obj_function, compute_grad_obj_function, W0, gamma, 100)
t2_proj = time.time()
len_proj = t2_proj - t1_proj

print('The projected gradient method returns: ' + str(min_g_proj))

The projected gradient method returns : 442.2878543960478


<h2> 4 - Algorithmic refinement for the problem with $H_{0}$ fixed </h2>

<h3> Question 4.1 </h3>

Implement a line search to the projected gradient method, in order to free ourselves from the need of a known Lipschitz constant.

In [24]:
g_init = compute_obj_function(vectorize(W0, H0))
print('The initial value of g is: ' + str(g_init))

The initial value of g is : 456.31969468823496


In [26]:
def taylor_line_search(val_g, grad_g, W, gamma):
    a, b = 0.5, 2 * gamma
    i = 0
    gamma2 = b * (a**i)
    x = W - gamma2 * unvectorize_M(grad_g(vectorize(W, H0)))[0]
    while (val_g(vectorize(x, H0)) > 
           val_g(vectorize(W, H0)) - (gamma2/2) * np.linalg.norm(unvectorize_M(grad_g(vectorize(W, H0)))[0])**2):
        gamma2 *= a
        x = W - gamma2 * unvectorize_M(grad_g(vectorize(W, H0)))[0]
    return gamma2
    
def projected_gradient_line_search_method(val_g, grad_g, W0, N):
    gamma = 1
    x = W0
    for k in range(N):
        gamma = taylor_line_search(val_g, grad_g, x, gamma)
        x = np.maximum(0, x - gamma * unvectorize_M(grad_g(vectorize(x, H0)))[0])
    return val_g(vectorize(x, H0))

In [27]:
t1_line = time.time()
min_g_line = projected_gradient_line_search_method(compute_obj_function, compute_grad_obj_function, W0, 100)
t2_line = time.time()
len_line = t2_line - t1_line

print('The projected gradient method with line search returns: ' + str(min_g_line))

The projected gradient method with line search returns : 445.4267145889815


<h3> Question 4.2 </h3>

Compare the performance of both algorithms.

In [22]:
print('The initialization gives us: g = ' + str(g_init) + '\n\n' +
      'With the projected gradient method, we have: g = ' + str(min_g_proj) + ', obtained in : ' + str(len_proj) + ' s. \n\n' +
      'With the line search gradient method, we have: g = ' + str(min_g_line) + ', obtained in : ' + str(len_line) + ' s. \n\n')

The initialization gives us : g = 456.31969468823496

With the projected gradient method, we have : g = 442.2878543960478, obtained in : 0.18251252174377441 s. 

With the line search gradient method, we have : g = 445.4267145889815, obtained in : 1.1948065757751465 s. 




<h2> 5 - Resolution of the full problem </h2>

<h3> Question 5.1 </h3>

Solve Problem (1) by the projected gradient method with line search for $N = 1000$ iterations. What does the algorithm return?

In [16]:
def full_taylor_line_search(val_g, grad_g, W_H, gamma):
    a, b = 0.5, 2 * gamma
    i = 0
    gamma2 = b * (a**i)
    x = W_H - gamma2 * grad_g(W_H)
    while (val_g(x) > 
           val_g(W_H) - (gamma2/2) * np.linalg.norm(grad_g(W_H))**2):
        gamma2 *= a
        x = W_H - gamma2 * grad_g(W_H)
    return gamma2

def full_projected_gradient_method(val_g, grad_g, W0_H0, N):
    gamma = 1
    x_y = W0_H0
    
    for k in range(N) :
        gamma = full_taylor_line_search(val_g, grad_g, x_y, gamma)
        x_y = np.maximum(0, x_y - gamma * grad_g(x_y))
    
    W, H = unvectorize_M(x_y)
    return W, H, val_g(x_y)

In [30]:
W_f, H_f, fproj = full_projected_gradient_method(compute_obj_function, compute_grad_obj_function, vectorize(W0, H0), 1000)

print('The projected gradient method with line search applied to both variables returns: ' + str(fproj))

The projected gradient method with line search applied to both variables returns : 358.9223041984623


As shown in the cell above, the result is better than before.

<h3> Question 5.2 </h3>

When $W$ (resp. $H$) is fixed, the problem is easier to solve. The method of alternate minimizations uses this fact and consists in the following method:

> <b>for</b> $t \geq 1$ <b>do</b> <br/>
$\,\,\,\, W_{t} \leftarrow \arg \min_{W} \frac{1}{2np} \left|\left| M - WH_{t-1} \right|\right|_{F}^{2}$ <br/>
$\,\,\,\, H_{t} \leftarrow \arg \min_{H} \frac{1}{2np} \left|\left| M - W_{t}H \right|\right|_{F}^{2}$<br/>
<b>end for</b>

Show that the value of the objective function is decreasing at each iteration. Deduce from this that the value converges.

Reminder: $g$ is defined by $g : (W, H) \mapsto \frac{1}{2np} \left|\left| M - WH \right|\right|_{F}^{2}$.

Let $t \geq 1$ and show that $g(W_{t}, H_{t}) \leq g(W_{t-1}, H_{t-1})$. 

By construction, for all $W \in \mathbb{R}^{n \times k}$, we have $g(W_{t}, H_{t-1}) \leq g(W, H_{t-1})$. In particular, we have $g(W_{t}, H_{t-1}) \leq g(W_{t-1}, H_{t-1})$.

Besides, for all $H \in \mathbb{R}^{k \times p}$, we have $g(W_{t}, H_{t}) \leq g(W_{t}, H)$. In particular, we have $g(W_{t}, H_{t}) \leq g(W_{t}, H_{t-1})$.

Then, by transitivity, we have: $$ g(W_{t}, H_{t}) \leq g(W_{t-1}, H_{t-1}) $$

<p style="text-align:center";><span style="border:1px solid black;padding:1%">$\left( g(W_{t}, H_{t}) \right)_{t \geq 1}$ is a decreasing suite, lower-bounded by $0$. Then, the suite converges.</span></p>

<h3> Question 5.3 </h3>

Code the alternate minimizations method.

In [18]:
def taylor_line_search_W(val_g, grad_g, W, H0, gamma):
    a, b = 0.5, 2 * gamma
    i = 0
    gamma2 = b * (a**i)
    x = W - gamma2 * unvectorize_M(grad_g(vectorize(W, H0)))[0]
    while (val_g(vectorize(x, H0)) > 
           val_g(vectorize(W, H0)) - (gamma2/2) * np.linalg.norm(unvectorize_M(grad_g(vectorize(W, H0)))[0])**2):
        gamma2 *= a
        x = W - gamma2 * unvectorize_M(grad_g(vectorize(W, H0)))[0]
    return gamma2

def projected_gradient_method_linesearch_W(val_g, grad_g, W0, H0, N):
    gamma = 1
    x = W0
    for k in range(N):
        gamma = taylor_line_search_W(val_g, grad_g, x, H0, gamma)
        x = np.maximum(0, x - gamma * unvectorize_M(grad_g(vectorize(x, H0)))[0])
    return x

def taylor_line_search_H(val_g, grad_g, W0, H, gamma):
    a, b = 0.5, 2 * gamma
    i = 0
    gamma2 = b * (a**i)
    y = H - gamma2 * unvectorize_M(grad_g(vectorize(W0, H)))[1]
    while (val_g(vectorize(W0, y)) > 
           val_g(vectorize(W0, H)) - (gamma2/2) * np.linalg.norm(unvectorize_M(grad_g(vectorize(W0, H)))[0])**2):
        gamma2 *= a
        y = H - gamma2 * unvectorize_M(grad_g(vectorize(W0, H)))[1]
    return gamma2

def projected_gradient_method_linesearch_H(val_g, grad_g, W0, H0, N):
    gamma = 1
    y = H0
    for k in range(N):
        gamma = taylor_line_search_H(val_g, grad_g, W0, y, gamma)
        y = np.maximum(0, y - gamma * unvectorize_M(grad_g(vectorize(W0, y)))[1])
    return y

def projected_alternate_minimization_method(val_g, grad_g, W0, H0, N):
    x = W0
    y = H0
    for k in range(N):
        x = projected_gradient_method_linesearch_W(val_g, grad_g, x, y, N//10)
        y = projected_gradient_method_linesearch_H(val_g, grad_g, x, y, N//10)
    return x, y, val_g(vectorize(x, y))

In [29]:
W_a, H_a, alternative_min = projected_alternate_minimization_method(compute_obj_function, compute_grad_obj_function, W0, H0, 100)

print('The alternate minimizations method returns: ' + str(alternative_min)) 

The alternate minimizations method returns : 358.939133356636


<h3> Question 5.4 </h3>

Compare projected gradient and alternate minimizations methods. Are the solutions the same? Is the objective value the same? How do the computing times compare?

In [20]:
N = 100

t1_f = time.time()
W_f, H_f, min_g_fproj = full_projected_gradient_method(compute_obj_function, compute_grad_obj_function, vectorize(W0, H0), N)
t2_f = time.time()
len_f = t2_f - t1_f

t1_a = time.time()
W_a, H_a, min_g_alternative = projected_alternate_minimization_method(compute_obj_function, compute_grad_obj_function, W0, H0, N)
t2_a = time.time()
len_a = t2_a - t1_a

In [21]:
print('With the projected method, we get: ' + '\n\n' +
      'W = ' + str(W_f) + '\n\n' +
      'H = ' + str(H_f) + '\n\n' + 
      'g(W, H) = ' + str(min_g_fproj) + '\n\n' +
      'in ' + str(len_f) + ' seconds' + '\n\n\n' +
      'With the alternate minimizations method, we get: ' + '\n\n' +
      'W = ' + str(W_a) + '\n\n' +
      'H = ' + str(H_a) + '\n\n' + 
      'g(W, H) = ' + str(min_g_alternative) + '\n\n' +
      'in ' + str(len_a) + ' seconds'
      )

With the projected method, we get : 

W = [[ 0.         71.87194646]
 [ 0.         78.95671828]
 [ 0.         73.78427977]
 [58.61889631 44.95150838]
 [42.82422019 54.7030421 ]
 [19.70616719 66.01665216]
 [15.67926382 65.83761374]
 [49.26104602 45.4162348 ]
 [69.34129776 36.40236423]
 [33.16588484 55.23490189]]

H = [[0.38022727 0.32407219 0.21108828 ... 0.26975117 0.14660154 0.15375377]
 [0.62086486 0.68036239 0.69322862 ... 0.70915623 0.71735043 0.75100897]]

g(W, H) = 363.5666859963321

in 0.556485652923584 seconds


With the alternate minimizations method, we get : 

W = [[  8.99411112  69.99680342]
 [  8.60837769  77.61389637]
 [  0.          76.51990801]
 [ 91.45289359  33.27808448]
 [ 78.98961077  40.74758491]
 [ 29.81289594  64.6332918 ]
 [ 31.26695766  61.36509863]
 [ 89.45646221  29.38187546]
 [113.30233658  19.25822191]
 [ 70.59605747  40.061465  ]]

H = [[0.33001953 0.30930524 0.24844045 ... 0.2449774  0.18905016 0.18328004]
 [0.59940083 0.65791425 0.67107203 ... 0.72638249

We remark that the alternate minimizations method gives a better minimum than the projection gradient method, but it is longer in terms of complexity (6.8s vs 0.6s). Besides, it shows that the solution obtained with the alternate minimizations method is more precise than the other one.

<h3> Question 5.5 </h3>

What stopping criterion could be used for the algorithms instead of just the number of iterations?

Instead of using a number of iterations, we could stop iterating when the norm of the difference between two successive values is less than a threshold that is fixed by the user (e.g. $\epsilon = 10^{-6}$).